<a href="https://colab.research.google.com/github/jhy9968/ECE6179_project/blob/main/Permutation_test_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Imports

In [ ]:
import os
import torch
import numpy as np
import pandas as pds
from tqdm import tqdm
from time import sleep
from scipy import optimize
from datetime import datetime
from sklearn import preprocessing
from torch.utils.data import Dataset
from matplotlib import pyplot as plt
from torch.utils.data import DataLoader
from torch.utils.data import random_split
from IPython.display import clear_output, display
from torch.utils.tensorboard import SummaryWriter
from torch.distributions.multivariate_normal import MultivariateNormal

# Load the TensorBoard notebook extension
%load_ext tensorboard

from google.colab import drive
drive.mount('/content/drive')

GPU_indx = 0
device = torch.device(GPU_indx if torch.cuda.is_available() else 'cpu')
print(device)

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
cpu


In [ ]:
# Load data
root_dir = '/content/drive/Shareddrives/Intersection_following_data/'
data = np.load(root_dir + 'data.npy', allow_pickle=True)

# Model, Dataset Class & Functions

In [ ]:
class LinearModel(torch.nn.Module):

  def __init__(self,input_dim=2,output_dim=2):
    super(LinearModel,self).__init__()

    self.fc1 = torch.nn.Linear(input_dim,output_dim)
    # self.log_sigma = torch.nn.Parameter(torch.ones(2,))
    self.log_sigma = torch.nn.Linear(input_dim,output_dim)
  
  def sample(self,x):
    dist = self.forward(x)
    return dist.sample()

  def get_stats(self,x):
    dist = self.forward(x)
    return dist.loc, dist.covariance_matrix

  def forward(self,x):
    mu = self.fc1(x)

    sigma = torch.exp(self.log_sigma(x))
    cov = torch.diag_embed(sigma)

    m = MultivariateNormal(mu,cov)
    return m

In [ ]:
class MyDataset(Dataset):
  def __init__(self, data, var_names, permutation=False):
    # Grab column names
    col_arr = data[0].columns.to_numpy()
    # Find the baseline input variables indexes and label indexes
    mask = np.array(['B' in col for col in col_arr])
    X_idx = np.array(range(len(mask)))[mask][:-2]
    Y_idx = np.array(range(len(mask)))[mask][-2:]
    self.input_names = col_arr[mask][:-2]
    self.output_names = col_arr[mask][-2:]
    # Find and append external input variables indexes
    for name in var_names:
      mask = np.array([name in col for col in col_arr])
      X_idx = np.append(X_idx, np.array(range(len(mask)))[mask])
      self.input_names = np.append(self.input_names, col_arr[mask])
    X_idx = np.sort(X_idx.astype('int32'))
    # Convert and concatenate all the data into one np array
    np_data = pds.concat(data,axis=0,ignore_index=True).to_numpy()

    self.X_idx = X_idx
    self.Y_idx = Y_idx

    self.X = np_data[:,X_idx]
    self.Y = np_data[:,Y_idx]
    # Randomly shuffle the labels for permutation test
    if permutation:
      self.Y = np.random.shuffle(self.Y)

  def __len__(self):
    return self.X.shape[0]

  def __getitem__(self, idx):
    x = self.X[idx,:]
    y = self.Y[idx,:]
    return x, y

  def get_idxs(self):
    return self.X_idx, self.Y_idx
  
  def in_out_dim(self):
    return self.X.shape[1], self.Y.shape[1]

  def in_out_names(self):
    return self.input_names, self.output_names

In [ ]:
def train_one_epoch(epoch_index, gpu=False):
    running_loss = 0.
    last_loss = 0.

    running_visual_loss = 0.
    last_visual_loss = 0.
    
    for inputs, labels in train_loader:
        inputs = torch.unsqueeze(inputs, dim=1).float()
        labels = labels.float()

        optimiser.zero_grad()
        if gpu:
          inputs = inputs.to(device)
          outputs = model(inputs)
          outputs.loc = outputs.loc.cpu()
          outputs.covariance_matrix = outputs.covariance_matrix.cpu()
        else:
          outputs = model(inputs)

        loss = loss_fn(outputs, labels)
        visual_loss = visual_loss_fn(torch.squeeze(outputs.loc), labels)
        loss.backward()

        optimiser.step()

        running_loss += loss.item()
        running_visual_loss += visual_loss.item()

    last_loss = running_loss / len(train_loader) # loss per batch
    last_visual_loss = running_visual_loss / len(train_loader)

    return last_loss, last_visual_loss


def loss_fn(y_pred, y_true):
  y_pred.loc = torch.squeeze(y_pred.loc)
  y_pred.covariance_matrix = torch.squeeze(y_pred.covariance_matrix)
  return torch.mean(-MultivariateNormal(y_pred.loc, y_pred.covariance_matrix).log_prob(y_true))

# Baseline

## Training

In [ ]:
batch_size = 256
baseline_set = MyDataset(data, var_names=[], permutation=True)

#Use 90% of data for training
num_train = int(len(baseline_set)*0.9)

#Split data into train/val sets
torch.manual_seed(0) #Set torch's random seed so that random split of data is reproducible
train_set, val_set = random_split(baseline_set, [num_train, len(baseline_set)-num_train])

train_loader = DataLoader(train_set, shuffle=True, batch_size=batch_size)
valid_loader = DataLoader(val_set, batch_size=batch_size)

print('Number of training data:', len(train_set))
print('Number of validation data:', len(val_set))
test_x, test_y = next(iter(train_loader))
print('x shape:', test_x.shape, '\ny shape:', test_y.shape)

Number of training data: 34614
Number of validation data: 3847
x shape: torch.Size([256, 48]) 
y shape: torch.Size([256, 2])


In [ ]:
EPOCHS = 4000
LEARNING_RATE = 1e-2
best_vloss = 1_000_000

if not os.path.isdir(root_dir + "Training/runs/"):
  os.makedirs(root_dir + "Training/runs/")
if not os.path.isdir(root_dir + "Training/models/"):
  os.makedirs(root_dir + "Training/models/")

timestamp = datetime.now().strftime('%Y%m%d%H%M%S')
writer = SummaryWriter(root_dir + 'Training/runs/permutation_baseline_{}_epochs_{}_lr_{}'.format(timestamp, EPOCHS, LEARNING_RATE))
epoch_number = 0

input_dim, output_dim = baseline_set.in_out_dim()
model = LinearModel(input_dim=input_dim,output_dim=output_dim)
if torch.cuda.is_available():
  model = model.to(device)
optimiser = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)
visual_loss_fn = torch.nn.MSELoss()

In [ ]:
%tensorboard --logdir=/content/drive/Shareddrives/Intersection_following_data/Training/runs

In [ ]:
min_avg_loss = 1e20
min_avg_visual_loss = 1e20
min_avg_vloss = 1e20
min_avg_visual_vloss = 1e20

for epoch in range(EPOCHS):

    # Training
    model.train(True)
    avg_loss, avg_visual_loss = train_one_epoch(epoch_number, gpu=torch.cuda.is_available())
    if avg_loss < min_avg_loss:
      min_avg_loss = avg_loss
    if avg_visual_loss < min_avg_visual_loss:
      min_avg_visual_loss = avg_visual_loss

    # Validation
    model.train(False)
    running_vloss = 0.0
    running_visual_vloss = 0.0
    running_vacc = 0.0
    with torch.no_grad():
      for i, vdata in enumerate(valid_loader):
          vinputs, vlabels = vdata
          vinputs = torch.unsqueeze(vinputs, dim=1).float()
          vlabels = vlabels.float()
          if torch.cuda.is_available():
            voutputs = model(vinputs.to(device))
            voutputs.loc = voutputs.loc.cpu()
            voutputs.covariance_matrix = voutputs.covariance_matrix.cpu()
          else:
            voutputs = model(vinputs)
          vloss = loss_fn(voutputs, vlabels)
          visual_vloss = visual_loss_fn(torch.squeeze(voutputs.loc), vlabels)
          running_vloss += vloss
          running_visual_vloss += visual_vloss
    avg_vloss = running_vloss / (i + 1)
    avg_visual_vloss = running_visual_vloss / (i + 1)
    if avg_vloss < min_avg_vloss:
      min_avg_vloss = avg_vloss
    if avg_visual_vloss < min_avg_visual_vloss:
      min_avg_visual_vloss = avg_visual_vloss

    # Print loss
    if epoch_number % int(EPOCHS/10) == int(EPOCHS/10)-1:
      print('EPOCH {}: LOSS train {} min. train {} valid {}'.format(epoch_number+1, avg_loss, min_avg_loss, avg_vloss))
    
    # Log loss
    writer.add_scalars('Training vs. Validation Loss',
                    { 'Training' : avg_loss, 'Validation' : avg_vloss},
                    epoch_number + 1)
    writer.add_scalars('Training vs. Validation MSE Loss',
                    { 'Training' : avg_visual_loss, 'Validation' : avg_visual_vloss},
                    epoch_number + 1)
    writer.add_scalars('Training vs. Validation Min Loss',
                    { 'Training' : min_avg_loss, 'Validation' : min_avg_vloss},
                    epoch_number + 1)
    writer.add_scalars('Training vs. Validation Min MSE Loss',
                    { 'Training' : min_avg_visual_loss, 'Validation' : min_avg_visual_vloss},
                    epoch_number + 1)
    writer.flush()

    # Track best performance, and save the model's state
    if avg_vloss < best_vloss:
        best_vloss = avg_vloss
        model_path = root_dir + 'Training/models/permutation_best_baseline{}'.format(timestamp)
        torch.save(model.state_dict(), model_path)

    epoch_number += 1

## Testing

In [ ]:
input_dim, output_dim = baseline_set.in_out_dim()
best_model = LinearModel(input_dim=input_dim,output_dim=output_dim)
# timestamp = "20221013223324"
best_model.load_state_dict(torch.load(root_dir + 'Training/models/permutation_best_baseline{}'.format(timestamp)))

<All keys matched successfully>

In [ ]:
in_names, out_names = baseline_set.get_idxs()
root_dir='/content/drive/Shareddrives/Intersection_following_data/'

row = 10
col = 4
fig, axs = plt.subplots(row,col, figsize=(col*5,row*5))

entropy_list_baseline = []

for data_id in range(row):

  test_x = torch.from_numpy((data[data_id].to_numpy()[:,in_names])).float()
  test_y = torch.from_numpy((data[data_id].to_numpy()[:,out_names])).float()
  # test_x = torch.unsqueeze(test_x, dim=1).float()
  pred_y = best_model(test_x)

  test_y = torch.squeeze(test_y).detach().numpy()
  pred_y.loc = torch.squeeze(pred_y.loc)
  pred_y.covariance_matrix = torch.squeeze(pred_y.covariance_matrix)
  pred_y_loc = pred_y.loc.detach().numpy()

  # SEs = np.square(np.linalg.norm((test_y-pred_y_loc), axis=1))

  # samples = []
  # for i in range(pred_y.loc.shape[0]):
  #   multinorm = MultivariateNormal(loc=pred_y.loc[i], covariance_matrix=pred_y.covariance_matrix[i])
  #   for j in range(10):
  #     samples.append(multinorm.sample().detach().numpy())
  # samples = np.stack(samples)

  entropy = []
  for i in range(pred_y.loc.shape[0]):
    multinorm = MultivariateNormal(loc=pred_y.loc[i], covariance_matrix=pred_y.covariance_matrix[i])
    entropy.append(multinorm.entropy().detach().numpy())
  entropy = np.stack(entropy)
  entropy_list_baseline.append(entropy)

#   axs[data_id, 0].plot(test_y[:,0], test_y[:,1], 'blue', linewidth=2, label='True trajectory')
#   axs[data_id, 0].plot(pred_y_loc[:,0], pred_y_loc[:,1], 'red', linewidth=2, label='Prediction')
#   axs[data_id, 0].set_aspect('equal')

#   axs[data_id, 1].scatter(samples[:,0], samples[:,1], s=5, c='gray', alpha=0.2, label='Samples')
#   axs[data_id, 1].plot(test_y[:,0], test_y[:,1], 'blue', linewidth=2, label='True trajectory')
#   axs[data_id, 1].plot(pred_y_loc[:,0], pred_y_loc[:,1], 'red', linewidth=2, label='Prediction')
#   axs[data_id, 1].set_aspect('equal')

#   axs[data_id, 2].plot(entropy, label='Entropy')
#   axs[data_id, 2].set_ylim(-8.0, -5.5)

#   axs[data_id, 3].plot(SEs, label='Square Error')
#   axs[data_id, 3].set_ylim(0.0, 0.0004)

# axs[0,0].legend()
# axs[0,1].legend()
# axs[0,2].legend()
# axs[0,3].legend()

# plt.show()

entropy_data = np.array(entropy_list_baseline, dtype=object)
np.save(root_dir+"entropy_data/permutation_baseline_entropy.npy", entropy_data)

# All-In

## Training

In [ ]:
batch_size = 256
all_in_set = MyDataset(data, var_names=["A"], permutation=True)

#Use 90% of data for training
num_train = int(len(all_in_set)*0.9)

#Randomly split data into train/val sets
torch.manual_seed(0) #Set torch's random seed so that random split of data is reproducible
train_set, val_set = random_split(all_in_set, [num_train, len(all_in_set)-num_train])

train_loader = DataLoader(train_set, shuffle=True, batch_size=batch_size)
valid_loader = DataLoader(val_set, batch_size=batch_size)

print('Number of training data:', len(train_set))
print('Number of validation data:', len(val_set))
test_x, test_y = next(iter(train_loader))
print('x shape:', test_x.shape, '\ny shape:', test_y.shape)

Number of training data: 34614
Number of validation data: 3847
x shape: torch.Size([256, 96]) 
y shape: torch.Size([256, 2])


In [ ]:
EPOCHS = 4000
LEARNING_RATE = 1e-2
best_vloss = 1_000_000

if not os.path.isdir(root_dir + "Training/runs/"):
  os.makedirs(root_dir + "Training/runs/")
if not os.path.isdir(root_dir + "Training/models/"):
  os.makedirs(root_dir + "Training/models/")

timestamp = datetime.now().strftime('%Y%m%d%H%M%S')
writer = SummaryWriter(root_dir + 'Training/runs/permutation_all_in_{}_epochs_{}_lr_{}'.format(timestamp, EPOCHS, LEARNING_RATE))
epoch_number = 0

input_dim, output_dim = all_in_set.in_out_dim()
model = LinearModel(input_dim=input_dim,output_dim=output_dim)
if torch.cuda.is_available():
  model = model.to(device)
  print("Using GPU")
optimiser = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)
visual_loss_fn = torch.nn.MSELoss()
print(timestamp)

Using GPU
20221024042212


In [ ]:
%tensorboard --logdir=/content/drive/Shareddrives/Intersection_following_data/Training/runs

In [ ]:
min_avg_loss = 1e20
min_avg_visual_loss = 1e20
min_avg_vloss = 1e20
min_avg_visual_vloss = 1e20

for epoch in range(EPOCHS):

    # Training
    model.train(True)
    avg_loss, avg_visual_loss = train_one_epoch(epoch_number, gpu=torch.cuda.is_available())
    if avg_loss < min_avg_loss:
      min_avg_loss = avg_loss
    if avg_visual_loss < min_avg_visual_loss:
      min_avg_visual_loss = avg_visual_loss

    # Validation
    model.train(False)
    running_vloss = 0.0
    running_visual_vloss = 0.0
    running_vacc = 0.0
    with torch.no_grad():
      for i, vdata in enumerate(valid_loader):
          vinputs, vlabels = vdata
          vinputs = torch.unsqueeze(vinputs, dim=1).float()
          vlabels = vlabels.float()
          if torch.cuda.is_available():
            voutputs = model(vinputs.to(device))
            voutputs.loc = voutputs.loc.cpu()
            voutputs.covariance_matrix = voutputs.covariance_matrix.cpu()
          else:
            voutputs = model(vinputs)
          vloss = loss_fn(voutputs, vlabels)
          visual_vloss = visual_loss_fn(torch.squeeze(voutputs.loc), vlabels)
          running_vloss += vloss
          running_visual_vloss += visual_vloss
    avg_vloss = running_vloss / (i + 1)
    avg_visual_vloss = running_visual_vloss / (i + 1)
    if avg_vloss < min_avg_vloss:
      min_avg_vloss = avg_vloss
    if avg_visual_vloss < min_avg_visual_vloss:
      min_avg_visual_vloss = avg_visual_vloss

    # Print loss
    if epoch_number % int(EPOCHS/10) == int(EPOCHS/10)-1:
      print('EPOCH {}: LOSS train {} min. train {} valid {}'.format(epoch_number+1, avg_loss, min_avg_loss, avg_vloss))
    
    # Log loss
    writer.add_scalars('Training vs. Validation Loss',
                    { 'Training' : avg_loss, 'Validation' : avg_vloss},
                    epoch_number + 1)
    writer.add_scalars('Training vs. Validation MSE Loss',
                    { 'Training' : avg_visual_loss, 'Validation' : avg_visual_vloss},
                    epoch_number + 1)
    writer.add_scalars('Training vs. Validation Min Loss',
                    { 'Training' : min_avg_loss, 'Validation' : min_avg_vloss},
                    epoch_number + 1)
    writer.add_scalars('Training vs. Validation Min MSE Loss',
                    { 'Training' : min_avg_visual_loss, 'Validation' : min_avg_visual_vloss},
                    epoch_number + 1)
    writer.flush()

    # Track best performance, and save the model's state
    if avg_vloss < best_vloss:
        best_vloss = avg_vloss
        model_path = root_dir + 'Training/models/permutation_best_all_in_{}'.format(timestamp)
        torch.save(model.state_dict(), model_path)

    epoch_number += 1

## Testing

In [ ]:
input_dim, output_dim = all_in_set.in_out_dim()
# timestamp = "20221024042212"
best_model = LinearModel(input_dim=input_dim,output_dim=output_dim)
best_model.load_state_dict(torch.load(root_dir + 'Training/models/permutation_best_all_in_{}'.format(timestamp)))

<All keys matched successfully>

In [ ]:
in_names, out_names = all_in_set.get_idxs()

row = 49
col = 4
# fig, axs = plt.subplots(row,col, figsize=(col*5,row*5))

entropy_list_all_in = []

for data_id in range(row):

  test_x = torch.from_numpy((data[data_id].to_numpy()[:,in_names])).float()
  test_y = torch.from_numpy((data[data_id].to_numpy()[:,out_names])).float()
  # test_x = torch.unsqueeze(test_x, dim=1).float()
  pred_y = best_model(test_x)

  test_y = torch.squeeze(test_y).detach().numpy()
  pred_y.loc = torch.squeeze(pred_y.loc)
  pred_y.covariance_matrix = torch.squeeze(pred_y.covariance_matrix)
  pred_y_loc = pred_y.loc.detach().numpy()

  # SEs = np.square(np.linalg.norm((test_y-pred_y_loc), axis=1))

  # samples = []
  # for i in range(pred_y.loc.shape[0]):
  #   multinorm = MultivariateNormal(loc=pred_y.loc[i], covariance_matrix=pred_y.covariance_matrix[i])
  #   for j in range(10):
  #     samples.append(multinorm.sample().detach().numpy())
  # samples = np.stack(samples)

  entropy = []
  for i in range(pred_y.loc.shape[0]):
    multinorm = MultivariateNormal(loc=pred_y.loc[i], covariance_matrix=pred_y.covariance_matrix[i])
    entropy.append(multinorm.entropy().detach().numpy())
  entropy = np.stack(entropy)
  entropy_list_all_in.append(entropy)

#   axs[data_id, 0].plot(test_y[:,0], test_y[:,1], 'blue', linewidth=2, label='True trajectory')
#   axs[data_id, 0].plot(pred_y_loc[:,0], pred_y_loc[:,1], 'red', linewidth=2, label='Prediction')
#   axs[data_id, 0].set_aspect('equal')

#   axs[data_id, 1].scatter(samples[:,0], samples[:,1], s=5, c='gray', alpha=0.2, label='Samples')
#   axs[data_id, 1].plot(test_y[:,0], test_y[:,1], 'blue', linewidth=2, label='True trajectory')
#   axs[data_id, 1].plot(pred_y_loc[:,0], pred_y_loc[:,1], 'red', linewidth=2, label='Prediction')
#   axs[data_id, 1].set_aspect('equal')

#   axs[data_id, 2].plot(entropy, label='Entropy')
#   axs[data_id, 2].set_ylim(-7.0, -5.0)

#   axs[data_id, 3].plot(SEs, label='Square Error')

# axs[0,0].legend()
# axs[0,1].legend()
# axs[0,2].legend()
# axs[0,3].legend()

# plt.show()

entropy_data = np.array(entropy_list_all_in, dtype=object)
np.save(root_dir+"entropy_data/permutation_all_in_entropy.npy", entropy_data)

# Cross Validation

## Training

In [ ]:
batch_size = 256
var_name = "head_Av"
model_name = var_name+"xy"
print(model_name)
CV_set = MyDataset(data, var_names=[var_name+"x",var_name+"y"], permutation=True)

#Use 90% of data for training
num_train = int(len(CV_set)*0.9)

#Split data into train/val sets
torch.manual_seed(0) #Set torch's random seed so that random split of data is reproducible
train_set, val_set = random_split(CV_set, [num_train, len(CV_set)-num_train])

train_loader = DataLoader(train_set, shuffle=True, batch_size=batch_size)
valid_loader = DataLoader(val_set, batch_size=batch_size)

print('Number of training data:', len(train_set))
print('Number of validation data:', len(val_set))
test_x, test_y = next(iter(train_loader))
print('x shape:', test_x.shape, '\ny shape:', test_y.shape)

head_Avxy
Number of training data: 34614
Number of validation data: 3847
x shape: torch.Size([256, 56]) 
y shape: torch.Size([256, 2])


In [ ]:
EPOCHS = 4000
LEARNING_RATE = 1e-2
best_vloss = 1_000_000

if not os.path.isdir(root_dir + "Training/runs/"):
  os.makedirs(root_dir + "Training/runs/")
if not os.path.isdir(root_dir + "Training/models/"):
  os.makedirs(root_dir + "Training/models/")

timestamp = datetime.now().strftime('%Y%m%d%H%M%S')
writer = SummaryWriter(root_dir + 'Training/runs/permutation_{}_{}_epochs_{}_lr_{}'.format(model_name, timestamp, EPOCHS, LEARNING_RATE))
epoch_number = 0

input_dim, output_dim = CV_set.in_out_dim()
model = LinearModel(input_dim=input_dim,output_dim=output_dim)
if torch.cuda.is_available():
  model = model.to(device)
optimiser = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)
visual_loss_fn = torch.nn.MSELoss()
print(timestamp)

20221025062411


In [ ]:
%tensorboard --logdir=/content/drive/Shareddrives/Intersection_following_data/Training/runs

In [ ]:
min_avg_loss = 1e20
min_avg_visual_loss = 1e20
min_avg_vloss = 1e20
min_avg_visual_vloss = 1e20

for epoch in range(EPOCHS):

    # Training
    model.train(True)
    avg_loss, avg_visual_loss = train_one_epoch(epoch_number, gpu=torch.cuda.is_available())
    if avg_loss < min_avg_loss:
      min_avg_loss = avg_loss
    if avg_visual_loss < min_avg_visual_loss:
      min_avg_visual_loss = avg_visual_loss

    # Validation
    model.train(False)
    running_vloss = 0.0
    running_visual_vloss = 0.0
    running_vacc = 0.0
    with torch.no_grad():
      for i, vdata in enumerate(valid_loader):
          vinputs, vlabels = vdata
          vinputs = torch.unsqueeze(vinputs, dim=1).float()
          vlabels = vlabels.float()
          if torch.cuda.is_available():
            voutputs = model(vinputs.to(device))
            voutputs.loc = voutputs.loc.cpu()
            voutputs.covariance_matrix = voutputs.covariance_matrix.cpu()
          else:
            voutputs = model(vinputs)
          vloss = loss_fn(voutputs, vlabels)
          visual_vloss = visual_loss_fn(torch.squeeze(voutputs.loc), vlabels)
          running_vloss += vloss
          running_visual_vloss += visual_vloss
    avg_vloss = running_vloss / (i + 1)
    avg_visual_vloss = running_visual_vloss / (i + 1)
    if avg_vloss < min_avg_vloss:
      min_avg_vloss = avg_vloss
    if avg_visual_vloss < min_avg_visual_vloss:
      min_avg_visual_vloss = avg_visual_vloss

    # Print loss
    if epoch_number % int(EPOCHS/10) == int(EPOCHS/10)-1:
      print('EPOCH {}: LOSS train {} min. train {} valid {}'.format(epoch_number+1, avg_loss, min_avg_loss, avg_vloss))
    
    # Log loss
    writer.add_scalars('Training vs. Validation Loss',
                    { 'Training' : avg_loss, 'Validation' : avg_vloss},
                    epoch_number + 1)
    writer.add_scalars('Training vs. Validation MSE Loss',
                    { 'Training' : avg_visual_loss, 'Validation' : avg_visual_vloss},
                    epoch_number + 1)
    writer.add_scalars('Training vs. Validation Min Loss',
                    { 'Training' : min_avg_loss, 'Validation' : min_avg_vloss},
                    epoch_number + 1)
    writer.add_scalars('Training vs. Validation Min MSE Loss',
                    { 'Training' : min_avg_visual_loss, 'Validation' : min_avg_visual_vloss},
                    epoch_number + 1)
    writer.flush()

    # Track best performance, and save the model's state
    if avg_vloss < best_vloss:
        best_vloss = avg_vloss
        model_path = root_dir + 'Training/models/permutation_best_{}_{}'.format(model_name, timestamp)
        torch.save(model.state_dict(), model_path)

    epoch_number += 1

## Testing

In [ ]:
input_dim, output_dim = CV_set.in_out_dim()
# timestamp = "20221025062411"
best_model = LinearModel(input_dim=input_dim,output_dim=output_dim)
best_model.load_state_dict(torch.load(root_dir + 'Training/models/permutation_best_{}_{}'.format(model_name, timestamp), map_location=torch.device('cpu')))

<All keys matched successfully>

In [ ]:
in_names, out_names = CV_set.get_idxs()

row = 49
col = 4
# fig, axs = plt.subplots(row,col, figsize=(col*5,row*5))

entropy_list_cv = []

for data_id in range(row):

  test_x = torch.from_numpy((data[data_id].to_numpy()[:,in_names])).float()
  test_y = torch.from_numpy((data[data_id].to_numpy()[:,out_names])).float()
  # test_x = torch.unsqueeze(test_x, dim=1).float()
  pred_y = best_model(test_x)

  test_y = torch.squeeze(test_y).detach().numpy()
  pred_y.loc = torch.squeeze(pred_y.loc)
  pred_y.covariance_matrix = torch.squeeze(pred_y.covariance_matrix)
  pred_y_loc = pred_y.loc.detach().numpy()

  # SEs = np.square(np.linalg.norm((test_y-pred_y_loc), axis=1))

  # samples = []
  # for i in range(pred_y.loc.shape[0]):
  #   multinorm = MultivariateNormal(loc=pred_y.loc[i], covariance_matrix=pred_y.covariance_matrix[i])
  #   for j in range(10):
  #     samples.append(multinorm.sample().detach().numpy())
  # samples = np.stack(samples)

  entropy = []
  for i in range(pred_y.loc.shape[0]):
    multinorm = MultivariateNormal(loc=pred_y.loc[i], covariance_matrix=pred_y.covariance_matrix[i])
    entropy.append(multinorm.entropy().detach().numpy())
  entropy = np.stack(entropy)
  entropy_list_cv.append(entropy)

#   axs[data_id, 0].plot(test_y[:,0], test_y[:,1], 'blue', linewidth=2, label='True trajectory')
#   axs[data_id, 0].plot(pred_y_loc[:,0], pred_y_loc[:,1], 'red', linewidth=2, label='Prediction')
#   axs[data_id, 0].set_aspect('equal')

#   axs[data_id, 1].scatter(samples[:,0], samples[:,1], s=5, c='gray', alpha=0.2, label='Samples')
#   axs[data_id, 1].plot(test_y[:,0], test_y[:,1], 'blue', linewidth=2, label='True trajectory')
#   axs[data_id, 1].plot(pred_y_loc[:,0], pred_y_loc[:,1], 'red', linewidth=2, label='Prediction')
#   axs[data_id, 1].set_aspect('equal')

#   axs[data_id, 2].plot(entropy, label='Entropy')

#   axs[data_id, 3].plot(SEs, label='Square Error (mm\u00b2)')

# axs[0,0].legend()
# axs[0,1].legend()
# axs[0,2].legend()
# axs[0,3].legend()

# plt.show()

entropy_data = np.array(entropy_list_cv, dtype=object)
np.save(root_dir+"entropy_data/permutation_cv_{}_entropy.npy".format(var_name), entropy_data)